# Decorator(Function Closure)

## Decorators wrap existing callables
- A decorator is callable that takes an callable as arguments and produces an enhanced callable as a result

In [3]:
def foo(bar: 'callable') -> 'callable':
    def inner(*args, **kwargs):
        print('before executing bar')
        bar(*args, **kwargs)
        print('after executing bar')
    return inner

@foo
def bar():
    print('executing bar')
    
bar()

before executing bar
executing bar
after executing bar


## @ syntax
- @ indicates the callable is wrapped by decorator
```
@f
@g
def h():
    pass
```
is equivalent to
```
k = f(g(h))
```

## Preserving function metadata
- As ```k = f(g(h))```, the useful metadata of h can be lost, docstring or name or etc

In [8]:
def f(g: "f's input") -> "f's output":
    """f's docstring"""
    def inner(*args: "f's inner args", **kwargs: "f's inner kwargs") -> "f's inner output":
        """f's inner's docstring"""
        return g(*args, **kwargs)
    return inner

@f
def g(*args: "g's args", **kwargs: "g's kwargs") -> "g's output":
    """g's docstring"""
    print("g's message")

print(g.__qualname__)
print(g.__annotations__)
print(g.__doc__)

f.<locals>.inner
{'args': "f's inner args", 'kwargs': "f's inner kwargs", 'return': "f's inner output"}
f's inner's docstring


- To preserve metadata, use ```functools.wraps(callable)```

In [9]:
from functools import wraps

def f(g: "f's input") -> "f's output":
    """f's docstring"""
    @wraps(g) # Preserve g's metadata
    def inner(*args: "f's inner args", **kwargs: "f's inner kwargs") -> "f's inner output":
        """f's inner's docstring"""
        return g(*args, **kwargs)
    return inner

@f
def g(*args: "g's args", **kwargs: "g's kwargs") -> "g's output":
    """g's docstring"""
    print("g's message")

print(g.__qualname__)
print(g.__annotations__)
print(g.__doc__)

g
{'args': "g's args", 'kwargs': "g's kwargs", 'return': "g's output"}
g's docstring


## Decorator with parameters
- If a decorator needs arguments except for a callable, just passing parameters
- If decorator is called, ```@deco(parameters)```, the produced callable by ```deco``` is used for decorator

In [12]:
from functools import wraps
def print_num(n):
    def decorator(f):
        @wraps(f)
        def wrapper(*args, **kwargs):
            print('n is', n)
            return f(*args, **kwargs)
        return wrapper
    return decorator

@print_num(5)
def f():
    print("f's message")
f()

#Advanced

from functools import partial, wraps
# Decorator technique
def deco(f=None, n=1):
    if f == None:
        print('enter')
        return partial(deco, n=n)
    @wraps(f)
    def wrapper(*args, **kwargs):
        print('Calling decorated function')
        return f(*args, **kwargs)
    return wrapper

# deco(example1) executed
@deco
def example1():
    print('example1')

# deco(n=3) executed -> partial(deco, n=n)(example2) executed
@deco(n=3)
def example2():
    print('example2')
    
example1()
example2()

n is 5
f's message
enter
Calling decorated function
example1
Calling decorated function
example2


## Chaining decorators
```
def outer1(fn):
    def inner1():
        return fn()
    return inner1
   
def outer2(fn):
    def inner2():
        return fn()
    return inner2
    
@outer1
@outer2
def fn():
    return
```
is queued by definition order in **definition step**
```
(front) - outer2 - outer1 - (end)
```
```
(front) - outer1 - (end) #pop outer2
```
```
(front) - (end) #pop outer1
```
is stacked by calling order in **calling step**
```
 (top)                         (top)                         (top)                       (top)
 inner1                        inner2                          |                           |
   |                             |                             fn        ==>(pop fn)    (bottom)
 inner2    ==>(pop inner1)       fn        ==>(pop inner2)     |
   |                             |                          (bottom)
   fn                         (bottom)
(bottom)
```

- According to the python version, **decorator can be cached**, so its order maybe defferent

In [16]:
def deco1(fn):
    print("deco1")
    def inner1():
        print("inner1")
        return fn()
    return inner1
def deco2(fn):
    print("deco2")
    def inner2():
        print("inner2")
        return fn()
    return inner2

@deco1
@deco2
def fn():
    print("fn")
    
print("===========")
fn()

deco2
deco1
inner1
inner2
fn


## Class Decorators
- When needing to maintain some state about the different functions it is wrapping, it is good to maintain states in namespace, that is, **class**
- Not mean decorating **class**
- Class must be **Callable**. so implement ```__call__```
- The decorated callable becomes the instance of decorator class

In [122]:
class countcalls(object):
    __instances = {} # To save instances is a default pattern
    
    def __init__(self, f): #Decorated callable is passed at initializer
        print('init', f.__name__)
        self.__f = f
        self.__numCalls = 0
        countcalls.__instances[f] = self #key is raw callable. value is decorated callable
    def __call__(self, *args, **kwargs): #Arguments of decorated callable is passed at call
        print('call')
        self.__numCalls += 1
        return self.__f(*args, **kwargs)
    def count(self):
        return self.__numCalls
    @staticmethod
    def counts():
        return dict([
            (f.__name__, countcalls.__instances[f].__numCalls) for f in countcalls.__instances # loop is in keys
        ])
            
@countcalls
def f():
    pass

@countcalls
def g():
    pass

assert isinstance(f, countcalls)
assert isinstance(g, countcalls)

for _ in range(2):
    f()
    g()
print(f.count())
print(countcalls.counts())

#Decorator with arguments
class A(object):
    def __init__(self, value = 5): # Decorator's arguments is here
        print("init A")
        print("value is", value)
    def __call__(self, fn): #Decorated function is here
        print("call A")

        def decorator(n): # Decorated function's arguments is here
            fn(n)
        return decorator


# Class decorator consumes the decorated callable as its arguments.
# So, just writing A means to initialize object like A(decorated_func)
# As the result, decorated_func is an object of A
@A
def decorated_func(n):
    print("Hello!")
print("---------------")
print(decorated_func)

print("---------------")

# Writing A with argument, A(1), means to initialize A with arguments and then execute object.__call__
# The execution of class decorator ends when decorated_func is reached
# Below executions is A(1) -> A(1).__call__(decorated_func) -> decorated_func = decorator(returned by __call__)
@A(1)
def decorated_func(n):
    print("Hello!")

print("---------------")
decorated_func(5)

init f
init g
call
call
call
call
2
{'f': 2, 'g': 2}
init A
value is <function decorated_func at 0x107f15f30>
---------------
---------------
init A
value is 1
call A
---------------
Hello!


## Decorate Class
- To decorate classes is similar to decorating functions
- To decorate the initializer, use it same as fucntions

In [50]:
# function decorator
def addSay(person:'Person') -> 'Person':
    def say(self):
        print("hello")
    if 'say' not in vars(person):
        person.say = say
    return person

@addSay
class Person:
    def __init__(self, name):
        self.name = name
        
assert 'say' in vars(Person)
Person("yyr").say()

# class decorator
class AddSay:
    def __init__(self, person): #Decorated callable is passed at initializer
        print("Init AddSay")
        self.__person = person
    def __call__(self, name): #Arguments of the decorated callable's initializer is passed at call
        print("call AddSay")
        def say(self):
            print("hello")
        
        self.__person.say = say
        newPerson = self.__person(name)
        return newPerson

@AddSay
class Person:
    def __init__(self, name):
        self.name = name
print("================")
assert isinstance(Person, AddSay) #Person is instance of AddSay
person = Person("yyr")
print("================")
person.say()

hello
Init AddSay
call AddSay
hello


In [4]:
from dataclasses import dataclass
from functools import wraps
from itertools import chain

def accept(*types):
    print('accept')
    def decorator(fn):
        print('decorator')
        @wraps(fn)
        def wrapper(*args, **kwargs):
            print('wrapper')
            for t, a in zip(iter(types), chain(iter(args), iter(kwargs.values()))):
                if not isinstance(a, t):
                    raise TypeError
            return fn(*args, **kwargs)
        return wrapper
    return decorator


@accept(float, float)
class Obj:
    def __init__(self, x, y):
        print('init')
    
    @accept()
    def __call__(self):
        print('call')
print("==============")
obj = Obj(3., 4.)
print("================")
obj()

print("================")

# So, in dataclass
@accept(float, float)
@dataclass
class Point:
    x:float
    y:float

print("================")
p = Point(3., 4.)

accept
accept
decorator
decorator
wrapper
init
wrapper
call
accept
decorator
wrapper


## Exercise

### Timing
- Decorator to time the execution of functions

In [61]:
import time
from functools import wraps
def timeit(fn):
    @wraps(fn)
    def wrapper(*args, **kwargs):
        start = time.time()
        retVal = fn(*args, **kwargs)
        end = time.time()
        interval = end - start
        print("%s : %2.2f sec" % (fn.__name__, interval))
        return retVal
    return wrapper


class TimeIt:
    __instances = {}
    
    def __init__(self, fn):
        self.__fn = fn
        self.__interval = 0.0
        TimeIt.__instances[fn] = self
    
    def __call__(self, *args, **kwargs):
        start = time.time()
        retVal = self.__fn(*args, **kwargs)
        end = time.time()
        self.__interval = end - start
        print("%s : %2.2f sec" %(self.__fn.__name__, self.__interval))
        return retVal
    
    @classmethod
    def get_intervals(cls):
        print("callable intervals")
        for fn in cls.__instances:
            print("%s : %2.2f sec" %(fn.__name__, cls.__instances[fn].__interval))
    
@timeit
def f():
    for _ in range(10**8):
        pass

@TimeIt
def f():
    for _ in range(10**8):
        pass

@TimeIt
def g():
    for _ in range(10**9):
        pass

f()
g()
TimeIt.get_intervals()

f : 1.00 sec
g : 9.41 sec
callable intervals
f : 1.00 sec
g : 9.41 sec


### Type-Checking
- Raise TypeError if arguments aren't matched

In [79]:
from itertools import chain
from functools import wraps
from itertools import zip_longest

def accept(*types):
    def decorator(fn):
        @wraps(fn)
        def wrapper(*args, **kwargs):
            for t, a in zip_longest(iter(types), chain(iter(args), iter(kwargs.values()))):
                if not isinstance(a, t):
                    raise TypeError
            return fn(*args, **kwargs)
        return wrapper
    return decorator

@accept(int, float)
def f(x:int, y:float):
    pass

f(1, y=3) 

TypeError: 

### Deprecation
- With the **warning** module, deprecated warning can be made
- To behave differently by version, use ```sys.version_info```

In [88]:
from sys import version_info
import warnings
from functools import wraps

def deprecated(fn):
    @wraps(fn)
    def wrapper(*args, **kwargs):
        if version_info.major > 2:
            warnings.warn("Deprecated: {}".format(fn.__name__), category=DeprecationWarning)
        return fn(*args, **kwargs)
    return wrapper

@deprecated
def f():
    pass
f()

/var/folders/qn/1t13bg7n0h1c7sv6dh677kq00000gn/T/ipykernel_52540/2579985892.py:9: DeprecationWarning: Deprecated: f
  warnings.warn("Deprecated: {}".format(fn.__name__), category=DeprecationWarning)


### Memorization
- Pure functions always returns the same results. So, the memorization is useful
- ```functools.lru_cache(maximum)``` is used to memorize the latest **maximum** number of function calls

In [93]:
from functools import wraps
# Memorize a factorial
def memorize(fn):
    cache = {}
    @wraps(fn)
    def wrapper(n):
        if n not in cache:
            cache[n] = fn(n)
        return cache[n]
    return wrapper

@memorize
def factorial(n):
    print("factorial {}".format(n))
    if n == 0:
        return 1
    return n * factorial(n-1)

factorial(5)
print('======================')

# General memorize(lru_cache)
class Memorize:
    __instances = {}
    def __init__(self, fn):
        self.__fn = fn
        self.__args = []
        self.__kwargs = {}
        self.__retVal = None
    
    def __call__(self, *args, **kwargs):
        if not self.__fn in Memorize.__instances:
            Memorize.__instances[self.__fn] = self
        if self.__args != args or self.__kwargs != kwargs:
            self.__args = args
            self.__kwargs = kwargs
            self.__retVal = self.__fn(*args, **kwargs)
        return self.__retVal
    
@Memorize
def factorial(n):
    print("factorial {}".format(n))
    if n == 0:
        return 1
    return n * factorial(n-1)

factorial(5)

factorial 5
factorial 4
factorial 3
factorial 2
factorial 1
factorial 0
factorial 5
factorial 4
factorial 3
factorial 2
factorial 1
factorial 0


120

### Print arguments

In [96]:
from functools import wraps
def print_args(fn):
    @wraps(fn)
    def wrapper(*args, **kwargs):
        print("args: {}".format(args))
        print("kwargs: {}".format(kwargs))
        return fn(*args, **kwargs)
    return wrapper

@print_args
def f(*args, **kwargs):
    pass
f(1,2,3, x=4, y=5)

args: (1, 2, 3)
kwargs: {'x': 4, 'y': 5}


### Assert when return None

In [97]:
from functools import wraps
def assert_None(fn):
    @wraps(fn)
    def wrapper(*args, **kwargs):
        retVal = fn(*args, **kwargs)
        assert retVal != None
        return retVal
    return wrapper

@assert_None
def f():
    return None
f()

AssertionError: 

### Restart functions

In [129]:
from functools import wraps
import time


class Restart:
    def __init__(self, maximum=5, sleep=1):
        self.__maximum = maximum
        self.__sleep = sleep
        self.__count = 0
        self.__fn = None
    def __call__(self, fn):
        self.__fn = fn
        @wraps(self.__fn)
        def wrapper(*args, **kwargs):
            try:
                return self.__fn(*args, **kwargs)
            except:
                self.__count += 1
                print("error count: {}".format(self.__count))
                if self.__count <= self.__maximum:
                    time.sleep(self.__sleep)
                    return wrapper(*args, **kwargs)
                raise
        return wrapper

@Restart(5, 1)
def f():
    raise RuntimeError
    
f()

error count: 1
error count: 2
error count: 3
error count: 4
error count: 5
error count: 6


RuntimeError: 